wikitable sortable jquery-tablesorter

In [1]:
from urllib.request import urlopen
import pandas as pd
#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

<b><h1>Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:<b></b></h1></b>

In [2]:
#specify the url
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
#Query the website and return the html to the variable 'page'
page = urlopen(wiki) 

In [4]:
#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page)
#soup.prettify

In [6]:
canda_table=soup.find('table', class_="wikitable sortable")
#canda_table

<b>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</b>

In [7]:
#Generate lists
Postcode=[]
Borough=[]
Neighbourhood=[]
for row in canda_table.findAll("tr"):
    row_cells=row.find_all('td')
    if len(row_cells)==3:
        Postcode.append(row_cells[0].find(text=True))
        Borough.append(row_cells[1].find(text=True))
        Neighbourhood.append(row_cells[2].find(text=True))


df=pd.DataFrame()
df['Postcode']=Postcode
df['Borough']=Borough
df['Neighbourhood']=Neighbourhood
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<b>Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.</b>

In [8]:
assigned_Borough=df[df.Borough!='Not assigned']
assigned_Borough.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


<b>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.</b>

In [9]:
after_Merge = assigned_Borough.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()


<b>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.</b>

In [10]:
after_Merge=after_Merge.replace('Not assigned\n',after_Merge[after_Merge.Neighbourhood.str.contains('Not assigned\n')].Borough.to_string())


<b> Cross check whether 'Not assigned is present in Neighbourhood or not'
    after_Merge[after_Merge.Neighbourhood.str.contains('Not assigned\n')==True]</b>

In [11]:
after_Merge[after_Merge.Neighbourhood.str.contains('Not assigned\n')==True]

,Postcode,Borough,Neighbourhood


In [12]:
after_Merge[after_Merge.Postcode=='M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,85 Queen's Park


<b>In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.</b>

In [13]:
after_Merge.shape

(103, 3)